# Cap rates

Compute a cap rate based on rents and recent single family home sales.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext ipy_blink1

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd

SCALE_FACTOR = 1.8031293436149882
VACANCY_RATE = 0.04
discount_rate_new = 0.08
discount_rate_ex = 0.04125
cap_rate = 0.0479
appreciation = 0.014
op_cost = 0.45

In [ ]:
# read paired estimated values and sales
sales =  pd.read_sql("""
SELECT "Main_SalesPriceAmount" AS price, "Main_RecordingDate" AS date, gid, total_rent, p.Building_PropertyLandUseStndCode
FROM diss.ztrans t
INNER JOIN diss.gp16 p ON (t."PropertyInfo_ImportParcelID" = p.Main_ImportParcelID)
WHERE SUBSTRING(t."Main_RecordingDate", 1, 4) IN ('2013', '2014', '2015', '2016', '2017')
WHERE Building_PropertyLandUseStndCode == 
""", "postgres://matthewc@localhost/matthewc")

In [ ]:
sales = sales.dropna(subset=['price', 'total_rent']).copy()
sales = sales[sales.price > 5000].copy()

In [ ]:
# make sure we only have SFHs here
assert (sales.building_propertylandusestndcode == 'RR101').all()

In [ ]:
# apply inflation using Zillow Home Value Index for LA, SFR only
# https://www.zillow.com/research/data/
zhvi = pd.read_csv('../data/Metro_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_mon.csv')
zhvi_la = zhvi.loc[zhvi.RegionName == 'Los Angeles-Long Beach-Anaheim, CA', [i for i in zhvi.columns if i.startswith('20')]].iloc[0]
# scale to appropriate scale factors
zhvi_la /= zhvi_la.loc['2017-12-31'] # this is the end of the ZTrans data and also the end of the PUMS data used to estimate rents

In [ ]:
# make sure this doesn't skip to the next month by mistake
assert str(pd.to_datetime('2012-07-02') + MonthEnd(0)).startswith('2012-07-31')
assert str(pd.to_datetime('2012-07-31') + MonthEnd(0)).startswith('2012-07-31')

In [ ]:
sales['date'] = pd.to_datetime(sales.date)

sales['month'] = (sales.date + MonthEnd(0)).astype(str)

sales.date.max()

In [ ]:
sales['zhvi'] = zhvi_la.loc[sales.month].values
assert not sales.zhvi.isnull().any()
sales['price17'] = sales.price / sales.zhvi

In [ ]:
sales['noi'] = sales.total_rent * (1 - op_cost) * 12 * SCALE_FACTOR * (1 - VACANCY_RATE)
sales['noi_to_value'] = sales.noi / sales.price17 * 100

In [ ]:
sales.noi_to_value.describe()

In [ ]:
# 2% trimmed mean
sales.loc[(sales.noi_to_value >= np.percentile(sales.noi_to_value, 2)) & (sales.noi_to_value <= np.percentile(sales.noi_to_value, 98)), 'noi_to_value'].mean()

In [ ]:
np.percentile(sales.noi_to_value, 2)

In [ ]:
np.percentile(sales.noi_to_value, 98)